In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.tree import DecisionTreeRegressor # 분석 알고리즘 DecisionTree 구현 라이브러리
from sklearn.linear_model import LinearRegression # 변동성이 약한 데이터에서는 강한 모습을 보인다.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split # 과거 데이터를 8:2나 7:3으로 자동으로 나눠주는 함수
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error # 분석 평가 대표지표 : MAE
from sklearn.metrics import mean_squared_error # 분석 평가 대표지표 : RMSE

# 데이터 불러오기

In [2]:
# 데이터 불러오기
carDataUrlPath ='https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/cars.csv'
carsData = pd.read_csv(carDataUrlPath)

In [3]:
carsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 963 entries, 0 to 962
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   age     963 non-null    int64
 1   gender  963 non-null    int64
 2   miles   963 non-null    int64
 3   debt    963 non-null    int64
 4   income  963 non-null    int64
 5   sales   963 non-null    int64
dtypes: int64(6)
memory usage: 45.3 KB


In [4]:
carsData

,age,gender,miles,debt,income,sales
0,28,0,23,0,4099,620
1,26,0,27,0,2677,1792
2,30,1,58,41576,6215,27754
3,26,1,25,43172,7626,28256
4,20,1,17,6979,8071,4438
...,...,...,...,...,...,...
958,22,0,11,8778,9829,1593
959,19,1,23,4850,3470,4742
960,28,1,28,9312,2720,12771
961,50,0,29,51343,8713,28511


# 성별 분리하기

In [5]:
# 남성
genderData0 = carsData.loc[carsData.gender == 0]

In [6]:
# 여성
genderData1 = carsData.loc[carsData.gender == 1]

In [7]:
genderData0.shape

(469, 6)

In [8]:
genderData1.shape

(494, 6)

# 남성 분석하기

In [9]:
genderData0Corr = genderData0.corr()

In [10]:
genderData0Corr

,age,gender,miles,debt,income,sales
age,1.000000,NaN,0.229790,0.240753,0.294120,0.393494
gender,NaN,NaN,NaN,NaN,NaN,NaN
miles,0.229790,NaN,1.000000,0.514677,0.444210,0.626935
debt,0.240753,NaN,0.514677,1.000000,0.491395,0.829762
income,0.294120,NaN,0.444210,0.491395,1.000000,0.690746
sales,0.393494,NaN,0.626935,0.829762,0.690746,1.000000


In [11]:
label = ['sales']

In [12]:
sortKey = list(genderData0Corr.loc[(abs(genderData0Corr.sales) > 0.5) & (genderData0Corr.sales != 1)].index) # ['miles', 'debt', 'income']

In [13]:
sortedData = genderData0.sort_values(sortKey, ignore_index=True)

In [15]:
sortedData

,age,gender,miles,debt,income,sales
0,48,0,10,0,188,971
1,23,0,10,1002,0,642
2,53,0,10,1777,1577,2336
3,26,0,10,1971,9754,5222
4,23,0,10,5330,3593,4756
...,...,...,...,...,...,...
464,45,0,92,59414,9996,29322
465,40,0,93,46003,11049,29652
466,32,0,95,12500,8644,28406
467,60,0,95,49927,10718,27591


In [16]:
standardYEARWEEK = 0.8

In [24]:
selectedIndex = int(sortedData.shape[0] * standardYEARWEEK)

In [25]:
selectedIndex

375

In [29]:
trainingDataSort0 = sortedData.loc[sortedData.index <= selectedIndex,sortKey]
trainingDataLabel0 = sortedData.loc[sortedData.index <= selectedIndex,label]

testDataSort0 = sortedData.loc[sortedData.index > selectedIndex,sortKey]
testDataLabel0 = sortedData.loc[sortedData.index > selectedIndex,label]

## 모델

In [26]:
model0 = DecisionTreeRegressor(random_state=10)

In [27]:
model0

DecisionTreeRegressor(random_state=10)

In [30]:
model0.fit(X=trainingDataSort0 ,y=trainingDataLabel0)

DecisionTreeRegressor(random_state=10)

In [31]:
predictValue = model0.predict(testDataSort0)

In [33]:
predictDF = pd.DataFrame(list(predictValue),columns=['predict'])

In [34]:
predictDF

,predict
0,22670.0
1,22670.0
2,5012.0
3,13699.0
4,7518.0
...,...
88,27229.0
89,22830.0
90,18655.0
91,22441.0


In [35]:
validateDf = pd.concat([testDataLabel0.reset_index(drop=True),predictDF],axis=1)

In [36]:
validateDf

,sales,predict
0,24545,22670.0
1,22854,22670.0
2,9977,5012.0
3,10780,13699.0
4,6025,7518.0
...,...,...
88,29322,27229.0
89,29652,22830.0
90,28406,18655.0
91,27591,22441.0


In [37]:
# MAE 평균절대오차
mae = mean_absolute_error(y_true=validateDf.sales,
                   y_pred=validateDf.predict)

# RMSE는 평균제곱근오차 MAE와 유사
# RMSE는 MAE에 비해 직관성은 떨어지지만, 로버스트한 성격(=극단적이지 않은)에서 강점을 보인다고 정리할 수 있다.
# "큰 오류값 차이에 대해서 크게 패널티를 주는" 이점이 있음
rmse = np.sqrt(mean_squared_error(y_true=validateDf.sales,
                   y_pred=validateDf.predict))
mae,rmse

(3039.451612903226, 4123.249132492742)

In [38]:
acc = pd.DataFrame(list(1-abs(validateDf['sales']-validateDf['predict'])/validateDf['sales']),columns=['acc3'])
realValidateDf3 = pd.concat([validateDf.reset_index(drop=True),acc],axis=1)
realValidateDf3

,sales,predict,acc3
0,24545,22670.0,0.923610
1,22854,22670.0,0.991949
2,9977,5012.0,0.502355
3,10780,13699.0,0.729221
4,6025,7518.0,0.752199
...,...,...,...
88,29322,27229.0,0.928620
89,29652,22830.0,0.769931
90,28406,18655.0,0.656727
91,27591,22441.0,0.813345


# 여성 분석하기

In [28]:
genderData1Corr = genderData1.corr()

In [29]:
genderData1Corr

,age,gender,miles,debt,income,sales
age,1.000000,NaN,0.235453,0.197007,0.184580,0.313312
gender,NaN,NaN,NaN,NaN,NaN,NaN
miles,0.235453,NaN,1.000000,0.576259,0.397049,0.646326
debt,0.197007,NaN,0.576259,1.000000,0.491015,0.841500
income,0.184580,NaN,0.397049,0.491015,1.000000,0.657815
sales,0.313312,NaN,0.646326,0.841500,0.657815,1.000000


In [30]:
label = ['sales']

In [31]:
sortKey2 = list(genderData1Corr.loc[(abs(genderData1Corr.sales) > 0.5) & (genderData1Corr.sales != 1)].index) 

In [32]:
sortedData1 = genderData1.sort_values(sortKey, ignore_index=True)

In [33]:
standard = 0.8

In [34]:
selectedIndex1 = int(sortedData1.shape[0] * standard)

In [35]:
trainingDataSort1 = sortedData1.loc[sortedData1.index <= selectedIndex1,sortKey2]
trainingDataLabel1 = sortedData1.loc[sortedData1.index <= selectedIndex1,label]

testDataSort1 = sortedData1.loc[sortedData1.index > selectedIndex1,sortKey2]
testDataLabel1 = sortedData1.loc[sortedData1.index > selectedIndex1,label]

In [36]:
model1 = DecisionTreeRegressor(random_state=10)

In [37]:
model1.fit(X=trainingDataSort1 ,y=trainingDataLabel1)

DecisionTreeRegressor(random_state=10)

In [38]:
predictValue1 = model0.predict(testDataSort1)

In [39]:
predictDF1 = pd.DataFrame(list(predictValue1),columns=['predict'])

In [40]:
validateDf1 = pd.concat([testDataLabel1.reset_index(drop=True),predictDF1],axis=1)

In [41]:
validateDf1

,sales,predict
0,6892,12021.0
1,12318,12151.0
2,10439,12021.0
3,15689,7135.0
4,19181,21758.0
...,...,...
93,29009,25157.0
94,29146,22441.0
95,27906,24486.0
96,29715,26689.0


In [42]:
# MAE 평균절대오차
mae = mean_absolute_error(y_true=validateDf1.sales,
                   y_pred=validateDf1.predict)

# RMSE는 평균제곱근오차 MAE와 유사
# RMSE는 MAE에 비해 직관성은 떨어지지만, 로버스트한 성격(=극단적이지 않은)에서 강점을 보인다고 정리할 수 있다.
# "큰 오류값 차이에 대해서 크게 패널티를 주는" 이점이 있음
rmse = np.sqrt(mean_squared_error(y_true=validateDf1.sales,
                   y_pred=validateDf1.predict))
mae,rmse

(3273.704081632653, 4521.612877083141)

In [43]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import joblib
sys.modules['sklearn.externals.joblib'] = joblib

In [44]:
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

In [45]:
dot_data = StringIO()
export_graphviz(model1, out_file=dot_data,
rounded=True,
special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

In [46]:
Image(graph.create_png())